# Методы оптимизации, ДЗ 2
## Сазанович Владислав М3439, Вариант 110

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go

import scipy
from scipy.misc import derivative

import pandas as pd

In [37]:
eps = 1e-9

LEFT = -10
RIGHT = 10

def f(x, y):
    return x**2 + y**2

def df(x0, y0):
    dx = scipy.misc.derivative(lambda x: f(x, y0), x0, eps)
    dy = scipy.misc.derivative(lambda y: f(x0, y), y0, eps)
    
    return (dx, dy)

In [18]:
X = np.linspace(-10, 10, 100)
Y = np.linspace(-10, 10, 100)
X, Y = np.meshgrid(X, Y)
Z = f(X, Y)

data = [
    go.Surface(
        x = X,
        y = Y,
        z = Z
    )
]
layout = go.Layout(
    title='f(x, y)',
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=65,
        r=50,
        b=65,
        t=90
    )
)
fig = go.Figure(data=data)
py.iplot(fig, filename='function')

# Метод покоординатного спуска

In [19]:
def golden_ratio(f):
    l = LEFT
    r = RIGHT
    
    # Значения функции в промежуточных точках неизвестны
    f1 = None
    f2 = None
    
    while (np.abs(r - l) / 2 > eps):
        m1 = l + (3 - np.sqrt(5)) / 2 * (r - l)
        m2 = l + (np.sqrt(5) - 1) / 2 * (r - l)
        
        # Вычисляем f если нет значения с предыдущего шага
        if f1 is None:
            f1 = f(m1)
            
        # Вычисляем f если нет значения с предыдущего шага
        if f2 is None:
            f2 = f(m2)
            
        if f1 <= f2:
            r = m2
            f2 = f1
            f1 = None
        else:
            l = m1
            f1 = f2
            f2 = None
    
    return (l + r) / 2

In [20]:
def coordinates_method(f):
    c_x = LEFT
    c_y = LEFT

    f_previous = None
    while (True):
        c_x = golden_ratio(lambda x: f(x, c_y))
        c_y = golden_ratio(lambda y: f(c_x, y))
        
        c_f = f(c_x, c_y)
        
        if ((f_previous is not None) and (np.abs(c_f - f_previous) < eps)):
            return c_x, c_y, c_f
        
        f_previous = c_f

In [21]:
coordinates_method(f)

(5.990735299878577e-16, 5.990735299878577e-16, 8.472179109675535e-16)

In [106]:
def gradient_method(f):
    alpha = 0.1
    c_x = LEFT
    c_y = LEFT
    
    f_previous = None
    
    cur_step = 0
    while (True):
        cur_step += 1
        
        if cur_step % 10 == 0:
            alpha = alpha * 0.9
        
        grad = df(c_x, c_y)
        
        c_x -= alpha * grad[0]
        c_y -= alpha * grad[1]
        c_f = f(c_x, c_y)
        
        if ((f_previous is not None) and (np.abs(c_f - f_previous) < eps)):
            return c_x, c_y, c_f

        f_previous = c_f

In [107]:
gradient_method(f)

(-4.9372225122448414e-05, -4.9372225122448414e-05, 4.875233227083453e-09)

### Сравнение алгоритмов

In [108]:
# Функция которая оборачивает f для подсчета количества ее вызовов.
def perf(function, invocations):
    # Добавляет 1 к счетчику и вызывает function
    def invoke_f(x, y, invocations):
        invocations[0] += 1
        return function(x, y)
    
    # Возвращаем фунцкию f которая будет дополнительно считать количество вызовов
    return lambda x, y: invoke_f(x, y, invocations) 

In [109]:
# Функция которая запускает алгоритм и пишет результат красиво
def measure_function(algo, algo_name):
    invocations = [0]
    x, y, с_f = algo(perf(f, invocations))
    
    print(
    """
    {}:
        Результат:
            x = {}
            y = {}
            z = {}
        Количество вызовов: {}
    """.format(algo_name, x, y, с_f, invocations[0]))

In [110]:
measure_function(coordinates_method, 'Покоординатный спуск')


    Покоординатный спуск:
        Результат:
            x = 5.990735299878577e-16
            y = 5.990735299878577e-16
            z = 7.177781886642253e-31
        Количество вызовов: 198
    


In [112]:
measure_function(gradient_method, 'Градиентный спуск')


    Градиентный спуск:
        Результат:
            x = -4.9372225122448414e-05
            y = -4.9372225122448414e-05
            z = 4.875233227083453e-09
        Количество вызовов: 80
    
